In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [14]:
data=pd.read_csv('MovieGenre.csv', encoding='latin-1')

In [15]:
data = data.drop(columns=['imdbId', 'Imdb Link', 'Poster'])

In [16]:
data = data.dropna(subset=["Genre", "IMDB Score", "Title"])

In [17]:
data.head()

,Title,IMDB Score,Genre
0,Toy Story (1995),8.3,Animation|Adventure|Comedy
1,Jumanji (1995),6.9,Action|Adventure|Family
2,Grumpier Old Men (1995),6.6,Comedy|Romance
3,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance
4,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance


In [18]:
data['Genre_list'] = data['Genre'].apply(lambda x: x.split('|'))

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(data['Genre_list'])

display(mlb.classes_)

array(['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport',
       'Talk-Show', 'Thriller', 'War', 'Western'], dtype=object)

In [20]:
X_text = data['Title']

In [21]:
X_score = data['IMDB Score'].values.reshape(-1, 1)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix


tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = tfidf.fit_transform(X_text)

scaler = StandardScaler()
X_score_scaled = scaler.fit_transform(X_score)

X_combined = hstack([X_tfidf, csr_matrix(X_score_scaled)])

In [23]:
from skmultilearn.model_selection import iterative_train_test_split

X_array = X_combined.toarray()

X_train, y_train, X_test, y_test = iterative_train_test_split(
    X_array, y, test_size=0.2
)

In [27]:
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score

In [28]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

model = MultiOutputClassifier(LogisticRegression(random_state=42))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Hamming Loss: {hamming_loss(y_test, y_pred):.4f}")
acc = accuracy_score(y_test, y_pred)
print("Subset Accuracy:", acc)

C:\Users\kakan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Hamming Loss: 0.0700
Subset Accuracy: 0.11066849246856716
